In [3]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from bayes_opt import BayesianOptimization


## Load & Shape Data

In [4]:
path = '~/Desktop/CareerFoundry/3.1./'

In [5]:

X_df = pd.read_pickle(os.path.join(path, 'Data/Clean/cleaned_for_keras.pkl'))
y_df = pd.read_csv(os.path.join(path, 'Data/Original/Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv')).drop(columns='DATE')


In [6]:

X = np.array(X_df).reshape(-1, 15, 9)
y_multi = np.array(y_df)
y = np.argmax(y_multi, axis=1)

## CNN Bayesian Optimization

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)


In [8]:
def build_cnn(filters, dense_units, dropout, lr):
    model = Sequential([
        Conv1D(filters=int(filters), kernel_size=2, activation='relu', input_shape=(15, 9)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(int(dense_units), activation='relu'),
        Dropout(float(dropout)),
        Dense(15, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(learning_rate=float(lr)),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
# objective function for fast run of BayesianOptimization
def objective(filters, dense_units, dropout, lr, batch_size):
    model = build_cnn(filters, dense_units, dropout, lr)

    es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

    hist = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=8,
        batch_size=int(batch_size),
        verbose=0,
        callbacks=[es]
    )
    return float(np.max(hist.history['val_accuracy']))


In [10]:
optimizer = BayesianOptimization(
    f=objective,
    pbounds={
        'filters': (16, 96),
        'dense_units': (32, 256),
        'dropout': (0.0, 0.5),
        'lr': (1e-4, 5e-3),
        'batch_size': (16, 128)
    },
    random_state=42
)

optimizer.maximize(init_points=3, n_iter=5)
best_params = optimizer.max['params']
best_params


|   iter    |  target   |  filters  | dense_... |  dropout  |    lr     | batch_... |
-------------------------------------------------------------------------------------


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 1         | 0.7975602 | 45.963209 | 244.96000 | 0.3659969 | 0.0030334 | 33.474087 |
| 2         | 0.7783909 | 28.479561 | 45.010729 | 0.4330880 | 0.0030454 | 95.304128 |
| 3         | 0.7882660 | 17.646759 | 249.25980 | 0.4162213 | 0.0011404 | 36.364396 |


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 4         | 0.8120824 | 47.990675 | 245.41735 | 0.3039147 | 0.0029454 | 33.291522 |


...

/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 7         | 0.8138251 | 50.498423 | 243.81566 | 0.2615462 | 0.0049810 | 36.779539 |


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


| 8         | 0.7708393 | 51.591136 | 239.17694 | 0.0       | 0.0001    | 35.346957 |


{'filters': np.float64(50.551398236143605),
 'dense_units': np.float64(243.83591253942257),
 'dropout': np.float64(0.38000436354957356),
 'lr': np.float64(0.003928825991051539),
 'batch_size': np.float64(32.75927343565979)}

In [12]:
# Train the final model using the best prams
final_model = build_cnn(
    filters=best_params['filters'],
    dense_units=best_params['dense_units'],
    dropout=best_params['dropout'],
    lr=best_params['lr']
)

es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

final_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=25,
    batch_size=int(best_params['batch_size']),
    verbose=2,
    callbacks=[es]
)


Epoch 1/25
431/431 - 1s - 3ms/step - accuracy: 0.6961 - loss: 0.9829 - val_accuracy: 0.7366 - val_loss: 0.7461
Epoch 2/25
431/431 - 1s - 2ms/step - accuracy: 0.7372 - loss: 0.7401 - val_accuracy: 0.7650 - val_loss: 0.6808
Epoch 3/25
431/431 - 1s - 1ms/step - accuracy: 0.7643 - loss: 0.6679 - val_accuracy: 0.7726 - val_loss: 0.6607
Epoch 4/25
431/431 - 1s - 1ms/step - accuracy: 0.7797 - loss: 0.6273 - val_accuracy: 0.7621 - val_loss: 0.6522
Epoch 5/25
431/431 - 1s - 1ms/step - accuracy: 0.7900 - loss: 0.5964 - val_accuracy: 0.8005 - val_loss: 0.5842
Epoch 6/25
431/431 - 1s - 1ms/step - accuracy: 0.8020 - loss: 0.5674 - val_accuracy: 0.8069 - val_loss: 0.5540
Epoch 7/25
431/431 - 1s - 1ms/step - accuracy: 0.8006 - loss: 0.5573 - val_accuracy: 0.7938 - val_loss: 0.5759
Epoch 8/25
431/431 - 1s - 1ms/step - accuracy: 0.8099 - loss: 0.5331 - val_accuracy: 0.8234 - val_loss: 0.5072
Epoch 9/25
431/431 - 1s - 1ms/step - accuracy: 0.8168 - loss: 0.5113 - val_accuracy: 0.8173 - val_loss: 0.5341
E

In [13]:
y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes=15) 

In [14]:
y_pred = np.argmax(final_model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
cm


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step


array([[3418,  207,   15,    3,    4,    2,    0,    1,    0,   31,    1,
           0,    0,    0],
       [ 169,  915,    2,    2,    0,    0,    0,    0,    0,    4,    0,
           0,    0,    0],
       [  32,   48,  121,    5,    0,    2,    0,    3,    0,    3,    0,
           0,    0,    0],
       [  18,    4,   16,   40,    1,    3,    0,    0,    0,    0,    0,
           0,    0,    0],
       [   4,    2,    4,    5,    2,    8,    0,    1,    0,    3,    0,
           0,    0,    0],
       [  10,    2,    5,    3,    1,   44,    0,    1,    0,   16,    0,
           0,    0,    0],
       [   5,    1,    1,    0,    1,    0,    0,    1,    0,    2,    0,
           0,    0,    0],
       [   8,    3,    3,    0,    0,    1,    0,   29,    0,   16,    0,
           1,    0,    0],
       [   7,    0,    0,    0,    0,    0,    0,    0,    0,    2,    0,
           0,    0,    0],
       [  25,   18,    8,    0,    0,   11,    0,    1,    0,  395,    0,
           0,    

In [15]:
test_loss, test_acc = final_model.evaluate(X_test, y_test, verbose=0)
test_acc


0.8661554455757141